# AI Workshop

Erstellt von [Pascal Sager](http://sagerpascal.github.io) und dem [ZHAW Zentrum für Künstliche Intelligenz](https://www.zhaw.ch/de/engineering/institute-zentren/cai/).

## Installation von Paketen

Nachfolgender Code installiert fünf Python-Pakete: `langchain`, `pypdf`, `numpy`, `openai`, `sentence_transformers`, und `faiss-cpu`. Diese Pakete bieten verschiedene Funktionen, die im Kontext von künstlicher Intelligenz (KI) und maschinellem Lernen (ML) relevant sind. Durch die Nutzung von Paketen können wir Code von anderen Entwicklern wiederverwenden und müssen nicht alles selbst schreiben. Dies ist ein wichtiger Aspekt der Softwareentwicklung, da es uns ermöglicht, schneller zu arbeiten und Fehler zu vermeiden.

##### Erklärung der Pakete

1. **`langchain`**: Ein Paket, das möglicherweise spezifische Funktionen im Bereich der Sprachverarbeitung oder Textanalyse bereitstellt.

2. **`pypdf`**: Ein Paket zur Bearbeitung von PDF-Dateien. Dies könnte in einem AI-Kontext nützlich sein, wenn Textinformationen aus PDFs extrahiert werden müssen.

3. **`numpy`**: Ein weit verbreitetes Paket für numerische Berechnungen in Python. Wird oft in Kombination mit AI-Anwendungen verwendet, insbesondere für die Handhabung von Datenarrays und Matrizen.

4. **`openai`**: Die OpenAI-Plattform bietet verschiedene AI-Modelle und Dienste an. Das Installieren des Pakets ermöglicht es, auf diese Ressourcen zuzugreifen und sie in eigenen Projekten zu verwenden.

5. **`sentence_transformers`**: Ein Paket, das prätrainierte Modelle für die Generierung von Vektoren aus Sätzen bereitstellt. Dies kann für Aufgaben wie semantische Ähnlichkeitsberechnungen nützlich sein.

6. **`faiss-cpu`**: Ein effizientes Indexierungs- und Suchpaket, das für das schnelle Durchsuchen großer Mengen von Vektoren optimiert ist. Kann in Anwendungen verwendet werden, die Ähnlichkeitsabfragen erfordern, wie sie in einigen AI- und ML-Szenarien vorkommen.


In [15]:
!pip install langchain
!pip install pypdf
!pip install numpy
!pip install openai
!pip install sentence_transformers
!pip install faiss-cpu
!pip install langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 29.1 MB/s eta 0:00:00



## Import von Modulen
Nach der Installation der Softwarepakete müssen diese importiert werden, um sie in Python verwenden zu können.
Der nachfolgende Code importiert verschiedene Module aus dem `langchain`-Paket und einige zusätzliche Module wie `pathlib` und `random`. Diese Module bieten Funktionen und Werkzeuge, die für die Bearbeitung von Textdaten, die Verarbeitung von PDF-Dateien, die Erstellung von Chatmodellen und die Arbeit mit Embeddings relevant sind.

##### Erklärung der Module

1. **`ChatOpenAI`**: Ein Modul aus `langchain` für Chatmodelle basierend auf der OpenAI-GPT-Architektur. Dies ermöglicht die Implementierung von Chatbots oder ähnlichen Anwendungen.

2. **`PromptTemplate`**: Ein Modul, das Vorlagen für Prompts (Aufforderungen) in Chatanwendungen bereitstellt. Prompts sind Anfragen oder Sätze, die an Chatmodelle gesendet werden, um eine spezifische Reaktion zu erhalten.

3. **`StrOutputParser`**: Ein Modul, das die Ausgaben von Chatmodellen oder anderen Textgeneratoren auf die Konsole ermöglicht.

4. **`RunnablePassthrough`**: Ein Modul, das für die Durchführung von Textoperationen oder -transformationen verwendet wird.

5. **`HumanMessage`**: Ein Modul, das für die Erstellung von Nachrichtenobjekten verwendet wird, die an Chatmodelle gesendet werden können.

6. **`BSHTMLLoader`**: Ein Modul, das Funktionen zum Laden von Text aus HTML-Dateien bereitstellt.

7. **`RecursiveCharacterTextSplitter`**: Ein Modul, das sich wahrscheinlich auf die Aufteilung von Text in Abschnitte oder Absätze konzentriert. Dies könnte in Verbindung mit der Verarbeitung von großen Textdaten nützlich sein.

8. **`HuggingFaceEmbeddings`**: Ein Modul für Embeddings, basierend auf Modellen von Hugging Face. Embeddings sind numerische Repräsentationen von Text, die in vielen AI-Anwendungen verwendet werden.

9. **`FAISS`**: Ein Modul für die Integration von FAISS, einem effizienten Indexierungs- und Suchpaket, das oft in Zusammenhang mit Vektoren und Ähnlichkeitsabfragen verwendet wird.

10. **`Path`**: Ein Modul aus der Standardbibliothek (`pathlib`), das Funktionen zum Arbeiten mit Dateipfaden bereitstellt.

11. **`random`**: Ein Modul aus der Standardbibliothek (`random`), das Funktionen für Zufallsoperationen bereitstellt.


In [16]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser, HumanMessage
from langchain.schema.runnable import RunnablePassthrough
from langchain_community.document_loaders import BSHTMLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain import FAISS
from pathlib import Path
import random

## Zugriff auf ein AI-Modell

Wir nutzen ein bereits trainiertes Modell von OpenAI, um unsere Applikation zu erstellen. Dieses Modell wurde mit einer großen Menge von Textdaten trainiert und kann daher auf viele verschiedene Fragen antworten. Wir können das Modell verwenden, um eine Frage zu stellen und eine Antwort zu erhalten.

Um Zugriff auf das Modell zu erhalten, müssen wir einen Zugriffscode (`key`) eingeben. Nachfolgender Code fragt uns nach diesem `key`, so dass wir ihn verwenden können, um Zugriff auf das Modell zu bekommen.


In [12]:
import getpass

print("Bitte gib den Zugriffscode ein:")
key = getpass.getpass()

Bitte gib den Zugriffscode ein:
··········


## Ziel

In diesem Workshop versuchen wir einen Chatbot zu erstellen, welchen wir Fragen über unsere Dokumente stellen können. Beispielsweise können wir Schulmaterial hochladen und der Chatbot sollte dann in der Lage sein, Fragen über dieses Material zu beantworten. Später können wir den Chatbot auch nutzen um Fragen zu den Dokumenten zu generieren, so dass wir uns auf eine Prüfung besser vorebereiten können.

## Fragen zu einem Dokument stellen

Wir beginnen sehr simpel und Fragen den Chatbot über unsere Dokumente aus. Nachfolgender Code fragt ein AI Modell was unsere Dokumente beinhalten. Wir können den Code ausführen und sehen, was passiert.


In [17]:
# Zuerst erstellen wir eine Verbindung zum Sprachmodell
llm = ChatOpenAI(openai_api_key=key, model_name="gpt-3.5-turbo")

# Dann sagen wir dem Modell, dass wir ein Mensch (Human) sind und was unsere Frage ist (content)
frage = HumanMessage(content="Was beinhalten meine Dokumente?")

# Danach können wir das Modell aufrufen
llm.invoke([frage])

AIMessage(content='Das kann ich nicht wissen, da ich keine Informationen über Ihre Dokumente habe. Sie können jedoch selbst überprüfen, welche Dokumente Sie besitzen und was sie enthalten. Falls Sie konkrete Fragen zu bestimmten Dokumenten haben, können Sie mir gerne mehr Details geben, damit ich Ihnen weiterhelfen kann.', response_metadata={'finish_reason': 'stop', 'logprobs': None})

Wie wir sehen kann der Chatbot unsere Frage nicht beantworten. Das liegt daran, dass er unsere Dokumente nicht kennt!

### Dokument erstellen

Damit es funktioniert, erstellen wir ein sehr einfaches Dokument und geben es zusammen mit der Frage in den Chatbot. Wir können den Code ausführen und sehen, was passiert.


In [18]:
mein_dokument = "Deep Learning (deutsch: mehrschichtiges Lernen, tiefes Lernen oder tiefgehendes Lernen) bezeichnet eine Methode des maschinellen Lernens, die künstliche neuronale Netze (KNN) mit zahlreichen Zwischenschichten (englisch hidden layers) zwischen Eingabeschicht und Ausgabeschicht einsetzt und dadurch eine umfangreiche innere Struktur herausbildet. Es ist eine spezielle Methode der Informationsverarbeitung."

frage = HumanMessage(content=f"Was beinhalten meine Dokumente? Mein Dokument hat folgenden Inhalt: {mein_dokument}")
antwort = llm.invoke([frage])

print("Die Frage ist: \n" + frage.content)
print("\n")
print("Die Antwort ist: \n" + antwort.content)

Die Frage ist: 
Was beinhalten meine Dokumente? Mein Dokument hat folgenden Inhalt: Deep Learning (deutsch: mehrschichtiges Lernen, tiefes Lernen oder tiefgehendes Lernen) bezeichnet eine Methode des maschinellen Lernens, die künstliche neuronale Netze (KNN) mit zahlreichen Zwischenschichten (englisch hidden layers) zwischen Eingabeschicht und Ausgabeschicht einsetzt und dadurch eine umfangreiche innere Struktur herausbildet. Es ist eine spezielle Methode der Informationsverarbeitung.


Die Antwort ist: 
Ihre Dokumente enthalten Informationen über Deep Learning, eine Methode des maschinellen Lernens, die künstliche neuronale Netze mit vielen Zwischenschichten zwischen Eingabe- und Ausgabeschicht verwendet. Diese Methode ermöglicht eine umfangreiche innere Struktur und ist eine spezielle Methode der Informationsverarbeitung.


Wie wir sehen kann der Chatbot unsere Frage beantworten. Das liegt daran, dass er unser Dokument nun kennt!

#### Frage generieren

Wir können den Chatbot auch nutzen um Fragen zu unseren Dokumenten zu generieren. Dazu gehen wir identisch vor, aber anstatt einer Zusammenfassung über den Inhalt verlangen wir eine Frage.


In [24]:
frage = HumanMessage(content=f"Erstelle eine spannende Frage über den Inhalt meines Dokuments. Mein Dokument hat folgenden Inhalt: {mein_dokument}")
antwort = llm.invoke([frage])

print("Unser Input ist: \n" + frage.content)
print("\n")
print("Die von der AI generierte Frage ist: \n" + antwort.content)

Unser Input ist: 
Erstelle eine spannende Frage über den Inhalt meines Dokuments. Mein Dokument hat folgenden Inhalt: Deep Learning (deutsch: mehrschichtiges Lernen, tiefes Lernen oder tiefgehendes Lernen) bezeichnet eine Methode des maschinellen Lernens, die künstliche neuronale Netze (KNN) mit zahlreichen Zwischenschichten (englisch hidden layers) zwischen Eingabeschicht und Ausgabeschicht einsetzt und dadurch eine umfangreiche innere Struktur herausbildet. Es ist eine spezielle Methode der Informationsverarbeitung.


Die von der AI generierte Frage ist: 
Wie können künstliche neuronale Netze durch Deep Learning mit zahlreichen Zwischenschichten eine umfangreiche innere Struktur herausbilden und welche Vorteile bringt dies für die Informationsverarbeitung?


## Nutzung von Templates

Bisher haben wir Fragen mit folgendem Code gestellt:

```python
HumanMessage(content=f"Was beinhalten meine Dokumente? Mein Dokument hat folgenden Inhalt: {mein_dokument}")
```

Diesen Code jedes mal zu schreiben ist aber ziemlich aufwendig, besonders wenn wir später mehrere Dokumente haben. Wir können den Code vereinfachen, indem wir ein Template verwenden. Ein Template ist eine Vorlage, welche wir mit Werten füllen können. Nachfolgender Code erstellt ein Template und füllt es mit Werten.


In [25]:
prompt_template = PromptTemplate.from_template(
    "Was beinhalten meine Dokumente? Die Dokumente haben folgenden Inhalt: {dokument}"
)

prompt = prompt_template.invoke({"dokument": "Mein Dokument Inhalt"})

print("Der Prompt hat folgenden Inhalt:\n" + prompt.text)

Der Prompt hat folgenden Inhalt:
Was beinhalten meine Dokumente? Die Dokumente haben folgenden Inhalt: Mein Dokument Inhalt


Anstatt den ganzen Text zu schreiben, können wir nun einfach das Template verwenden:

```python
prompt = prompt_template.invoke({"dokument": "Mein Dokument Inhalt"})
```



## Pipeline erstellen

Damit wir künftig noch weniger Arbeit haben, können wir eine Pipeline erstellen. Eine Pipeline ist eine Abfolge von Operationen, welche wir nacheinander ausführen.

Der Code wird nun schon etwas komplizierter. Wir erstellen eine Pipeline, welche die folgenden Schritte ausführt:

1. Wir generieren den Prompt mit unserem Template
2. Wir geben den Prompt an das AI Modell
3. Wir geben die Antwort des AI Modells an einen Parser, welcher die Antwort in einen lesbaren Text umwandelt

In [26]:
pipeline = prompt_template | llm | StrOutputParser()

Nun können wir die Pipeline verwenden, um eine Zusammenfassung über unsere Dokumente zu erhalten.
Dazu reicht eine Zeile Code:

```python
pipeline.invoke({"dokument": mein_dokument})
```

In [27]:
pipeline.invoke({"dokument": mein_dokument})

'Die Dokumente enthalten Informationen über Deep Learning, eine Methode des maschinellen Lernens, die künstliche neuronale Netze mit mehreren Zwischenschichten verwendet. Es wird erklärt, wie Deep Learning eine umfangreiche innere Struktur herausbildet und als spezielle Methode der Informationsverarbeitung dient.'

## Verwendung von Dokumenten

Bisher haben wir nur ein Dokument verwendet, welches wir direkt im Code programmiert hatten.

```python
mein_dokument = "Deep Learning (deutsch: mehrschichtiges Lernen, tiefes Lernen oder tiefgehendes Lernen) bezeichnet eine Methode des maschinellen Lernens, die künstliche neuronale Netze (KNN) mit zahlreichen Zwischenschichten (englisch hidden layers) zwischen Eingabeschicht und Ausgabeschicht einsetzt und dadurch eine umfangreiche innere Struktur herausbildet. Es ist eine spezielle Methode der Informationsverarbeitung."
```

Das ist ziemlich unpraktisch, da wir so nicht mit mehreren Dokumenten arbeiten können. Zudem müssen wir jedes mal den Code ändern, wenn wir ein Dokument hinzufügen oder löschen wollen.

Wir können den Code vereinfachen, indem wir alle Dokumente aus einem Ordner laden. So können wir künftig alle gewünschten Dokumente in diesen Ordner kopieren und der Chatbot wird sie automatisch verwenden.



### Dokumente herunterladen

Zur Demonstration laden wir Webseiten aus dem Internet als Daten herunter. Theoretisch könnten wir auch PDFs oder Word-Dateien verwenden - für den Computer sind alles Files die Text enthalten der unterschiedlich formatiert ist.

In [28]:
!wget https://de.wikipedia.org/wiki/Maschinelles_Lernen
!wget https://de.wikipedia.org/wiki/Deep_Learning

--2024-03-14 13:49:23--  https://de.wikipedia.org/wiki/Maschinelles_Lernen
Resolving de.wikipedia.org (de.wikipedia.org)... 208.80.154.224, 2620:0:861:ed1a::1
Connecting to de.wikipedia.org (de.wikipedia.org)|208.80.154.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 140507 (137K) [text/html]
Saving to: ‘Maschinelles_Lernen.1’

Maschinelles_Lernen 100%[===================>] 137.21K  --.-KB/s    in 0.004s  

2024-03-14 13:49:23 (31.9 MB/s) - ‘Maschinelles_Lernen.1’ saved [140507/140507]

--2024-03-14 13:49:23--  https://de.wikipedia.org/wiki/Deep_Learning
Resolving de.wikipedia.org (de.wikipedia.org)... 208.80.154.224, 2620:0:861:ed1a::1
Connecting to de.wikipedia.org (de.wikipedia.org)|208.80.154.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 117035 (114K) [text/html]
Saving to: ‘Deep_Learning.1’

Deep_Learning.1     100%[===================>] 114.29K  --.-KB/s    in 0.004s  

2024-03-14 13:49:23 (26.4 MB/s) - ‘Deep_Learning.

#### Dokumente öffnen

Wir finden nun also die HTML-Dokumente in dem Ordner. Nun müssen wir noch den Inhalt der Dokumente laden. Das können wir mit dem Paket `BSHTMLLoader` machen. Nachfolgender Code lädt den Inhalt dieser Dokumente.

In [37]:
import re
html_dokumente = ['Deep_Learning', 'Maschinelles_Lernen']

for html_dokument in html_dokumente:
    loader = BSHTMLLoader(html_dokument)
    data = loader.load()[0]
    data.page_content = re.sub(r'\n+', '\n', data.page_content).strip()
    print(f"Die Datei {html_dokument} beinhaltet \n\n ''{data.page_content[:200]}''  \n\n .... \n\n")

Die Datei Deep_Learning beinhaltet 

 ''Deep Learning – Wikipedia
Deep Learning
aus Wikipedia, der freien Enzyklopädie
Zur Navigation springen
Zur Suche springen
Geschichtete Repräsentation von Bildern auf mehreren Abstraktionsebenen[1]
Dee''  

 .... 


Die Datei Maschinelles_Lernen beinhaltet 

 ''Maschinelles Lernen – Wikipedia
Maschinelles Lernen
aus Wikipedia, der freien Enzyklopädie
Zur Navigation springen
Zur Suche springen
Maschinelles Lernen (ML) ist ein Fachgebiet, das statistische Algo''  

 .... 




#### Dokumente in Liste speichern

Wir haben nun also den Inhalt der Dokumente geladen. Nun müssen wir die Dokumente noch speichern, beispielsweise in einer Liste `dokumente`:

In [44]:
dokumente = []

for html_dokument in html_dokumente:
    loader = BSHTMLLoader(html_dokument)
    data = loader.load()[0]
    data.page_content = re.sub(r'\n+', '\n', data.page_content).strip()
    dokumente.append(data)

print(f"Wir haben insgesamt {len(dokumente)} Dokumente.")

Wir haben insgesamt 2 Dokumente.


#### Dokumente ausgeben

Nun können wir die Abschnitte anschauen indem wir auf die Liste zugreifen:

In [45]:
print(f"Dokument 1: {dokumente[0].page_content}\n\n")
print(f"Dokument 2: {dokumente[1].page_content}")

Dokument 1: Deep Learning – Wikipedia
Deep Learning
aus Wikipedia, der freien Enzyklopädie
Zur Navigation springen
Zur Suche springen
Geschichtete Repräsentation von Bildern auf mehreren Abstraktionsebenen[1]
Deep Learning (deutsch: mehrschichtiges Lernen, tiefes Lernen[2] oder tiefgehendes Lernen) bezeichnet eine Methode des maschinellen Lernens, die künstliche neuronale Netze (KNN) mit zahlreichen Zwischenschichten (englisch hidden layers) zwischen Eingabeschicht und Ausgabeschicht einsetzt und dadurch eine umfangreiche innere Struktur herausbildet.[3][4][5] Es ist eine spezielle Methode der Informationsverarbeitung.
Links: Eingangsschicht (input layer) mit in diesem Fall drei Eingangsneuronen. Rechts: Ausgabeschicht mit den Ausgangsneuronen, in diesem Bild zwei. Die mittlere Schicht wird als verborgen bezeichnet (hidden layer), da ihre Neuronen weder Eingänge noch Ausgänge sind. Hier ist nur eine verborgene Schicht zu sehen, aber viele Netzwerke haben deutlich mehr. Die notwendige A

#### Daten bereinigen

Wie wir feststellen, sehen die Abschnitte noch nicht sehr schön aus. Wir könnten sie bereinigen, indem wir Sonderzeichen entfernen, fehlende Abstände einfügen, etc. Dies wäre aber ein ziemlicher Aufwand und wir allen es in diesem Workshop weg. Wenn wir aber künftig eine noch bessere Chat-Qualität wollen, könnten wir dies noch nachholen.

## Grosse Datenmengen

Wir haben nun sehr viel Text eingelesen. Leider können AI Chatbots nur mit einer begrenzten Menge an Text arbeiten. Wir müssen also die Datenmenge reduzieren. Dies können wir machen, indem wir die Texte in kleinere Abschnitte aufteilen. Nachfolgender Code teilt die Texte in Abschnitte von 1000 Zeichen auf, wobei die Abschnitte sich um 200 Zeichen überlappen. Später wählen wir dann die "besten" Abschnitte aus und übergeben sie dem Chatbot.


In [51]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=150)
texte = text_splitter.split_documents(dokumente)
print(f"Wir haben unsere Dokumente in {len(texte)} Texte aufgeteilt.")

Wir haben unsere Dokumente in 167 Texte aufgeteilt.


### Passendste Texte finden

Denn passendsten Text direkt zu finden ist sehr schwierig. Wir können aber die passendsten Texte finden, indem wir die Texte mit einer Frage vergleichen. Dazu müssen wir die Texte in Vektoren umwandeln. Ein Vektor ist eine numerische Repräsentation eines Textes. Wir können dann die Ähnlichkeit zwischen der Frage und den Texten berechnen. Die Texte mit der höchsten Ähnlichkeit sollten dann die passendsten Texte sein.

Zur Umwandlung der Texte in Vektoren verwenden wir ein Embedding-Modell von Hugging Face. Dieses Modell wurde mit sehr vielen Texten trainiert und kann daher Texte in Vektoren umwandeln.

In [52]:
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L12-v2")

##### Embeddings

Nachfolgender Code wandelt einen Text in einen Vektor um. Wir können den Code ausführen und sehen, was passiert.

In [53]:
text = texte[1].page_content
embeddings = embedding_model.embed_query(text)
print(f"Der Text ist {text}\n\n")
print(f"Das dazugehörige Embedding besteht aus {len(embeddings)} Zahlen.\n\n")
print(f"Die Embeddings sind: {embeddings}\n\n")

Der Text ist Geschichtete Repräsentation von Bildern auf mehreren Abstraktionsebenen[1]
Deep Learning (deutsch: mehrschichtiges Lernen, tiefes Lernen[2] oder tiefgehendes Lernen) bezeichnet eine Methode des maschinellen Lernens, die künstliche neuronale Netze (KNN) mit zahlreichen Zwischenschichten (englisch hidden layers) zwischen Eingabeschicht und Ausgabeschicht einsetzt und dadurch eine umfangreiche innere Struktur herausbildet.[3][4][5] Es ist eine spezielle Methode der Informationsverarbeitung.


Das dazugehörige Embedding besteht aus 384 Zahlen.


Die Embeddings sind: [-0.014185012318193913, -0.03948082774877548, -0.05409707501530647, 0.08260055631399155, 0.0017176567343994975, 0.005088570527732372, -0.034705258905887604, 0.011471837759017944, -0.027410613372921944, 0.0015370505861938, -0.002251206897199154, -0.037350133061409, -0.011892452836036682, 0.006406981498003006, -0.024514252319931984, 0.054362352937459946, -0.05907692387700081, 0.14670012891292572, -0.00303934281691908

##### Ähnlichkeit berechnen

Wir können nicht nur unsere Texte in Vektoren umwandeln, sondern auch unsere Frage. Nachfolgender Code wandelt einige Fragen in Vektoren um.



In [58]:
embedded_frage_1 = embedding_model.embed_query("Ist tiefes Lernen eine Art des maschinellen Lernen?")
print(embedded_frage_1)
embedded_frage_2 = embedding_model.embed_query("Was ist die Eingangsschicht und die Ausgabeschicht?")
print(embedded_frage_2)
embedded_frage_3 = embedding_model.embed_query("Trinkt die Kuh Milch oder Wasser?")
print(embedded_frage_3)

[0.04565805196762085, 0.03119267337024212, 0.0034045418724417686, -0.007587556727230549, 0.09747792780399323, 0.016057834029197693, 0.06088406965136528, -0.0007447446696460247, -0.01949154958128929, 0.05100805312395096, 0.018386652693152428, -0.03461936116218567, -0.02988843433558941, 0.016160761937499046, 0.008028628304600716, 0.06239267811179161, 0.00575797026976943, 0.05787547677755356, 0.03145628795027733, 0.02753416635096073, 0.060553256422281265, 0.06716932356357574, 0.022254012525081635, 0.04500420391559601, 0.007381688803434372, -0.03401831537485123, -0.06774533540010452, 0.038843072950839996, -0.02074025385081768, -0.017202330753207207, 0.00772966630756855, -0.00279031740501523, -0.022588269785046577, 0.010966111905872822, 0.06083492934703827, -0.00010078286868520081, 0.04051096737384796, 0.01109327469021082, 0.01773117296397686, 0.03108968213200569, -0.052715547382831573, 0.04472431540489197, 0.02681240811944008, 0.0006068079965189099, 0.04354388639330864, 0.01326124556362629

Wir berechnen mithilfe der Kosinus-Ähnlichkeit einen Ähnlichkeitswert zwischen den Fragen und den Texten. Die Kosinus-Ähnlichkeit ist eine Metrik, welche die Ähnlichkeit zwischen zwei Vektoren berechnet. Je höher die Ähnlichkeit, desto ähnlicher sind sich die Vektoren. Nachfolgender Code berechnet die Ähnlichkeit zwischen den Fragen und den Texten.

In [59]:
import numpy as np

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

print(cosine_similarity(embeddings, embedded_frage_1))
print(cosine_similarity(embeddings, embedded_frage_2))
print(cosine_similarity(embeddings, embedded_frage_3))

0.44269228120719256
0.21822272423074607
0.07518472506192456


## Vektoren speichern

Das Berechnen der Vektoren ist sehr aufwendig. Wir können die Vektoren aber speichern, so dass wir sie nicht jedes mal neu berechnen müssen. Nachfolgender Code speichert die Vektoren unserer Texte in einer Datenbank, so dass wir künftig nur noch die Vektoren der Frage berechnen müssen.

Wir verwenden dazu das Paket `FAISS`. Dieses Paket beinhaltet eine sehr effiziente Datenbank die grosse Datenmengen speichern kann. Zudem kann es sehr schnell die Ähnlichkeit zwischen Vektoren berechnen.


In [60]:
datenbank = FAISS.from_documents(texte, embedding=embedding_model)
datenbank.save_local("data/index.faiss")

Die Texte sind nun in einer Datenbank gespeichert. Wir können sie nun laden und die Ähnlichkeit zwischen den Texten und einer Frage berechnen.

In [61]:
datenbank.similarity_search("Wozu dient die erste Schicht in einem neuronalen Netzwerk?")

[Document(page_content='benutzten künstlichen neuronalen Netze sind wie das menschliche Gehirn gebaut, wobei die Neuronen wie ein Netz miteinander verbunden sind. Die erste Schicht des neuronalen Netzes, die sichtbare Eingangsschicht, verarbeitet eine Rohdateneingabe, wie beispielsweise die einzelnen Pixel eines Bildes. Die Dateneingabe enthält Variablen, die der Beobachtung zugänglich sind, daher „sichtbare Schicht“.[3]', metadata={'source': 'Deep_Learning', 'title': 'Deep Learning – Wikipedia'}),
 Document(page_content='In der Regel werden die Neuronen in Schichten zusammengefasst. Die Signale wandern von der ersten Schicht (der Eingabeschicht) zur letzten Schicht (der Ausgabeschicht) und durchlaufen dabei möglicherweise mehrere Zwischenschichten (versteckte Schichten). Ein Netz wird als tiefes neuronales Netz bezeichnet, wenn es mindestens 2 versteckte Schichten hat. Darauf bezieht sich auch der Begriff Deep Learning. Jede Schicht kann die Signale an ihren Eingängen unterschiedlich 

## Zwischenfazit

Was wir bisher haben:

- Wir lesen alle Dokumente von einem Ordner ein
- Wir teilen die Dokumente in kleinere Abschnitte auf
- Wir wandeln die Abschnitte in Vektoren um
- Wir speichern die Vektoren in einer Datenbank
- Wir können die Ähnlichkeit zwischen den Vektoren und einer Frage berechnen

Was noch fehlt:
- Wir müssen die besten Abschnitte auswählen und dem Chatbot übergeben
- Wir wollen dem Chatbot eine Frage zum Dokument stellen und eine Antwort erhalten
- Der Chatbot soll eine Frage zum Dokument generieren und wir beantworten sie

Dazu bilden wir entsprechende Pipelines.


## Frage beantworten

Wir wollen nun eine Frage zum Dokument stellen und eine Antwort erhalten. Dazu müssen wir die Datenbank in die Pipeline einbauen. Die Funktion `datenbank.as_retriever()`erzeugt einen `Retriever`. Dieser Retriever kann für eine gegebene Frage die passendsten Texte finden.


In [62]:
retriever = datenbank.as_retriever()

Dieser Retriever kann wie folgt aufgerufen werden:

In [63]:
retriever.invoke("Was ist ein Neuron?")

[Document(page_content='Karl Steinbuchs Lernmatrix[12] war eines der ersten künstlichen neuronalen Netze, das aus mehreren Schichten von Lerneinheiten oder lernenden „Neuronen“ bestand. Damit war er einer der Wegbereiter des Deep Learning, bei dem es um tiefe neuronale Netze geht, die viele Aufgaben erlernen können, bei denen früheren einschichtige Perzeptronen scheitern.', metadata={'source': 'Deep_Learning', 'title': 'Deep Learning – Wikipedia'}),
 Document(page_content='anschließende Versuche besser zu steuern und somit mögliche Sackgassen in der Lösungsfindung frühzeitig zu verhindern.[17] Heute wird der Begriff jedoch vorwiegend im Zusammenhang mit künstlichen neuronalen Netzen verwendet und tauchte in diesem Kontext erstmals im Jahr 2000 auf, in der Veröffentlichung Multi-Valued and Universal Binary Neurons: Theory, Learning and Applications von Igor Aizenberg und Kollegen.[18][19][20]', metadata={'source': 'Deep_Learning', 'title': 'Deep Learning – Wikipedia'}),
 Document(page_c

#### Dokumente zusammensetzen

Wie wir sehen gibt der Retriever mehrere Texte zurück. Wir hängen diese Texte zusammen, so dass wir einen längeren Text erhalten, welchen wir direkt in das AI Modell geben können.

In [64]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

format_docs(retriever.invoke("Was ist ein Neuron?"))

'Karl Steinbuchs Lernmatrix[12] war eines der ersten künstlichen neuronalen Netze, das aus mehreren Schichten von Lerneinheiten oder lernenden „Neuronen“ bestand. Damit war er einer der Wegbereiter des Deep Learning, bei dem es um tiefe neuronale Netze geht, die viele Aufgaben erlernen können, bei denen früheren einschichtige Perzeptronen scheitern.\n\nanschließende Versuche besser zu steuern und somit mögliche Sackgassen in der Lösungsfindung frühzeitig zu verhindern.[17] Heute wird der Begriff jedoch vorwiegend im Zusammenhang mit künstlichen neuronalen Netzen verwendet und tauchte in diesem Kontext erstmals im Jahr 2000 auf, in der Veröffentlichung Multi-Valued and Universal Binary Neurons: Theory, Learning and Applications von Igor Aizenberg und Kollegen.[18][19][20]\n\nbenutzten künstlichen neuronalen Netze sind wie das menschliche Gehirn gebaut, wobei die Neuronen wie ein Netz miteinander verbunden sind. Die erste Schicht des neuronalen Netzes, die sichtbare Eingangsschicht, vera

#### Prompt Template

Wir geben im Prompt-Template eine eindeutige Anweisung, was der Chatbot tun soll. Nachfolgender Code erstellt ein Prompt-Template, welches den Chatbot anweist, eine Frage zu beantworten.

In [65]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Sie sind ein Assistent zur Beantwortung von Fragen. Beantworten Sie die Frage mit Hilfe der gegebenen Kontext-Informationen. Wenn Sie die Antwort nicht wissen, sagen Sie einfach, dass Sie es nicht wissen und erfinden Sie nichts. Verwenden Sie maximal drei Sätze und fassen Sie die Antwort kurz."),
        ("human", "Frage: {frage} \nKontext-Informationen: {kontext}\n"),
    ]
)

In [66]:
prompt_template.invoke({"kontext": "Gegebener Kontext", "frage": "Gegebene Frage"}).to_string()

'System: Sie sind ein Assistent zur Beantwortung von Fragen. Beantworten Sie die Frage mit Hilfe der gegebenen Kontext-Informationen. Wenn Sie die Antwort nicht wissen, sagen Sie einfach, dass Sie es nicht wissen und erfinden Sie nichts. Verwenden Sie maximal drei Sätze und fassen Sie die Antwort kurz.\nHuman: Frage: Gegebene Frage \nKontext-Informationen: Gegebener Kontext\n'

#### LLM Modell

Wir verwenden das LLM Modell von OpenAI, um die Frage zu beantworten. Dabei setzen wir die Temperatur auf 0, so dass wir eine präzisere Antwort erhalten.

In [67]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=key)

#### Pipeline

Wir setzen diese Bausteine nun in einer Pipeline zusammen. Nachfolgender Code erstellt eine Pipeline, welche basierend auf einer Frage die passendsten Texte findet und diese dem Chatbot übergibt. Der Chatbot gibt dann eine Antwort zurück.

In [68]:
rag_chain = (
    {"kontext": retriever | format_docs, "frage": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

Wir können die Pipeline nun verwenden, um eine Frage zu stellen und eine Antwort zu erhalten.

In [69]:
rag_chain.invoke("Was ist deep learning?")

'Deep Learning bezieht sich auf die Verwendung von künstlichen neuronalen Netzwerken mit vielen Schichten, um komplexe Muster in Daten zu erkennen und zu lernen. Es ermöglicht die automatische Extraktion von Merkmalen auf verschiedenen Abstraktionsebenen, insbesondere in Bereichen wie Bilderkennung und Sprachverarbeitung.'

In [70]:
frage = input("Frage: ")
print(rag_chain.invoke(frage))

Frage: Was ist ein neuronales Netzwerk?
Ein künstliches neuronales Netzwerk besteht aus einer Gruppe von Knoten, die miteinander verbunden sind, ähnlich wie Neuronen im menschlichen Gehirn. Es kann komplexe Informationen verarbeiten und lernen, Aufgaben wie Bilderkennung und Spracherkennung durchzuführen. Karl Steinbuchs Lernmatrix war eines der ersten künstlichen neuronalen Netze.


## Frage generieren

In [71]:
def get_random_dokument():
    random_document = random.choice(list(datenbank.docstore._dict.items()))
    return random_document

In [72]:
get_random_dokument()

('7061e809-e1a6-4bfe-8e8d-4525e0cbdcd3',
 Document(page_content='Karl Steinbuchs Lernmatrix[12] war eines der ersten künstlichen neuronalen Netze, das aus mehreren Schichten von Lerneinheiten oder lernenden „Neuronen“ bestand. Damit war er einer der Wegbereiter des Deep Learning, bei dem es um tiefe neuronale Netze geht, die viele Aufgaben erlernen können, bei denen früheren einschichtige Perzeptronen scheitern.', metadata={'source': 'Deep_Learning', 'title': 'Deep Learning – Wikipedia'}))

In [73]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Sie sind ein Assistent zur Generierung von Fragen. Generieren Sie eine neue Frage basierend auf den gegebenen Kontext-Informationen. Die Frage muss sehr spezifisch zum Inhalt sein und darf nicht generell sein wie 'was kommt im Dokument vor'. Generieren Sie einen Satz für die Frage mit einem Fragezeichen am Ende. Machen Sie danach 5 Leerzeilen und geben Sie mit maximal zwei Sätzen eine Musterlösung an. Beispielsweise könnten für die Kontext-Informationen 'Real Madrid ist ein Fussball Club' der Output wie folgt aussehen: 'FRAGE: Was ist Real Madrid? \n\n\n\n\nANTWORT: Real Madrid ist ein Fussball Club.'"),
        ("human", "Kontext-Informationen: {kontext}\n"),
    ]
)

In [74]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7, openai_api_key=key)

In [75]:
frage_chain = (
    {"kontext": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [76]:
print(frage_chain.invoke(get_random_dokument()[1].page_content))

FRAGE: Wer hat seit 1997 den vorwärtsgerichteten hierarchisch-konvolutionalen Ansatz durch seitliche und rückwärtsgerichtete Verbindungen erweitert?




ANTWORT: Sven Behnke hat seit 1997 in der Neuronalen Abstraktionspyramide den vorwärtsgerichteten hierarchisch-konvolutionalen Ansatz durch seitliche und rückwärtsgerichtete Verbindungen erweitert.
